## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
import re

from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return np.sum(l_c * (l_s - l_c) / l_s, axis=1) + np.sum(r_c * (r_s - r_c) / r_s, axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return - np.sum(l_c * (np.log(l_c) - np.log(l_s))) - np.sum(r_c * (np.log(r_c) - np.log(r_s))) 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_s - np.max(l_c) + r_s - np.max(r_c)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # sorts x and y along x, ascending
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # computes total number of classes
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода?
        # selects a subset of y between [min_samples_split, len(y)-min_samples_split)
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # finds "inflection points", i.e. indices, where y changes; 
        # adds min_samples_split + 1 to make them consistent with sorted_y
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # computes lengths of contiguous series of the same y value
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # converts class counts into sparse representation
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # adds data from excluded earlier to keep up with the original 
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split],
                                                                minlength=class_number)

        # Что делает этот блок кода?
        # computes class counts and split sizes for every sensible split
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # computes impurity with current impurity function for every split
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # retrieves the index least impure split
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # returns the least impure split and the threshold (mean value of target)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        def is_leaf():
            return x.shape[0] <= self.min_samples_split \
                   or self.max_depth and depth == self.max_depth \
                   or get_most_frequent_share(y) >= self.sufficient_share

        def get_most_frequent(arg):
            return np.argmax(np.bincount(arg))
        def get_most_frequent_count(arg): 
            return np.max(np.bincount(arg))
        def get_most_frequent_share(arg): 
            return get_most_frequent_count(arg) / arg.shape[0]

        def make_leaf(arg): 
            return (self.LEAF_TYPE, get_most_frequent(arg), get_most_frequent_share(arg))
        def make_non_leaf(*args): 
            return (self.NON_LEAF_TYPE, *args)

        if is_leaf():
            self.tree[node_id] = make_leaf(y)
            return

        feats = self.get_feature_ids(x.shape[1])
        imps = [(f, imp, thr) for f, imp, thr in ((f, *self.__find_threshold(x[:, f], y)) for f in feats) if thr]

        if not imps:
            self.tree[node_id] = make_leaf(y)
            return

        min_imp_feat, _, thr  = min(imps, key=lambda x: x[1])
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, min_imp_feat, thr)

        if not y_l.shape[0] and y_r.shape[0]:
            self.tree[node_id] = make_leaf(y_r)
        elif y_l.shape[0] and not y_r.shape[0]:
            self.tree[node_id] = make_leaf(y_l)
        elif y_l.shape[0] and y_r.shape[0]:
            self.tree[node_id] = make_non_leaf(min_imp_feat, thr)
            self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _, _ = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.39461207389831543
1.093472957611084


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=10, shuffle=True)

In [8]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.932817826557
0.933898727862
0.928660513844
0.931570632743
0.930489731438
0.935727945456
0.934231312879
0.933399850337
0.93132119398
0.934142690837


In [9]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.89481998836
0.890163798121
0.887669410493
0.888999750561
0.899975056124
0.891161553172
0.89639976719
0.890246944375
0.886172777916
0.888408448362


# Применить для задачи Titanic 

In [10]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_union, make_pipeline

In [11]:
df_train = pd.read_csv("train.csv", na_values="NaN", index_col=0)
df_test = pd.read_csv("test.csv", na_values="NaN", index_col=0)

In [12]:
class FeatureExtractor(TransformerMixin):

    def __init__(self, new_feature_name, extractor_function):
        self.new_feature_name = new_feature_name
        self.extractor_function = extractor_function
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X[self.new_feature_name] = self.extractor_function(X)
        return X

In [13]:
class MeanByCategoryImputer(TransformerMixin):

    def __init__(self, group_key, mean_key, nan_value=None):
        self.group_key = group_key
        self.mean_key = mean_key
        self.nan_value = nan_value
    
    def fit(self, X, y=None):
        self.means_by_cat = X.groupby(self.group_key).mean()[self.mean_key].to_dict()
        return self

    def transform(self, X, y=None):
        if self.nan_value:
            X[X[self.mean_key] == self.nan_value] = np.nan
        X[self.mean_key] = X[self.mean_key].fillna(X[self.group_key].map(self.means_by_cat))
        if sum(X[self.mean_key].isnull()) > 0: # we have a 1-member group
            X[self.mean_key] = X[self.mean_key].fillna(X[self.mean_key].mean())
        return X[[self.mean_key]]

In [14]:
class LabelEncoderPipelineFriendly(LabelEncoder):
    
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelEncoderPipelineFriendly, self).fit(X)
        
    def transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).transform(X).reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).fit(X).transform(X).reshape(-1, 1)

In [15]:
class FeaturesSum(TransformerMixin):
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return np.sum(X.astype(np.float64), axis=1).values.reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return self.transform(X)

In [16]:
def prepare_pipeline():
    def get_age_col(X):
        return X.copy()[["Age", "Name"]] #  mutation ahead
    
    def get_title(X):
        return X[["Name"]].apply(lambda x: re.match(".*\, ((the )?\S*)\. .*", x.Name).groups()[0], axis=1)
    
    def get_pclass_col(X):
        return X[["Pclass"]]
    
    def get_sex_col(X):
        return X["Sex"] #  LabelEncoder expects 1d array
    
    def get_sum_col(X):
        return X[["SibSp", "Parch"]]
    
    def get_ticket_prefix(X):
        def extract_prefix(x):
            match = re.match("(.*) .*", x.Ticket.replace(".", ""))
            if match or x.Ticket == "LINE":
                return 1
            return 0
        return X[["Ticket"]].apply(extract_prefix, axis=1).values.reshape(-1, 1)
    
    def get_cabin(X):
        return X["Cabin"].isnull().astype(int) #  LabelEncoder expects 1d array
    
    pipeline = make_union(*[
        make_pipeline(FunctionTransformer(get_pclass_col, validate=False), OneHotEncoder(sparse=False)),
        make_pipeline(FunctionTransformer(get_sex_col, validate=False), LabelEncoderPipelineFriendly()),
        make_pipeline(FunctionTransformer(get_age_col, validate=False),
                      FeatureExtractor("Title", get_title), 
                      MeanByCategoryImputer("Title", "Age"),
                      StandardScaler()),
        make_pipeline(FunctionTransformer(get_sum_col, validate=False), FeaturesSum(), StandardScaler()),
        make_pipeline(FunctionTransformer(get_ticket_prefix, validate=False), OneHotEncoder(sparse=False)),
        make_pipeline(MeanByCategoryImputer("Pclass", "Fare", 0.0), StandardScaler()),
        make_pipeline(FunctionTransformer(get_cabin, validate=False), LabelEncoderPipelineFriendly())
        
    ])
    return pipeline

In [17]:
x = prepare_pipeline().fit_transform(df_train)
y = np.array(df_train.Survived)

In [18]:
my_clf = MyDecisionTreeClassifier(min_samples_split=5)
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.544444444444
0.831460674157
0.539325842697
0.707865168539
0.797752808989
0.76404494382
0.584269662921
0.775280898876
0.573033707865
0.730337078652


In [19]:
clf = DecisionTreeClassifier(min_samples_split=5)
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.855555555556
0.831460674157
0.808988764045
0.831460674157
0.786516853933
0.786516853933
0.820224719101
0.775280898876
0.76404494382
0.820224719101


In [20]:
preds = my_clf.predict(prepare_pipeline().fit_transform(df_test))

In [21]:
result = pd.DataFrame({"PassengerId": df_test.index, "Survived": preds})

In [22]:
result.to_csv("submission.csv", sep=",", index=False)